In [162]:
import numpy as np
import pandas as pd
import math
df=pd.read_csv("PS4-1/PS4_1_X_Train.csv")
X = df.values
# pd.read_csv("PS4-1/PS4_2_Y_Test.csv")
y_train=pd.read_csv("PS4-1/PS4_2_Y_Train.csv")
# y = y_train.iloc[:, -1].values.reshape(-1,1)
y = y_train.values
len(y)
num_of_test_data=len(X)
gap=math.ceil(num_of_test_data/20)
X=X[::gap]
y=y[::gap]

In [163]:
df=pd.read_csv("PS4-1/PS4_1_X_Train.csv")
y_train=pd.read_csv("PS4-1/PS4_2_Y_Train.csv")
df['target']=y_train['0']

In [164]:
df=df.sort_values(by="-0.21961")
X=df['-0.21961'].values
X=X.reshape(-1,1)
y=df['target'].values
y=y.reshape(-1,1)

In [165]:
num_of_test_data=len(X)
gap=math.ceil(num_of_test_data/20)
X=X[::gap]
y=y[::gap]

In [169]:
df.loc[df['-0.21961']==-0.17345]

,-0.21961,2.0415,target
20,-0.17345,2.7476,0


In [167]:
y

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1]])

In [153]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        ''' constructor ''' 
        
        # for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        
        # for leaf node
        self.value = value
class DecisionTreeClassifier():
    def __init__(self, min_samples_split=2, max_depth=2):
        ''' constructor '''
        
        # initialize the root of the tree 
        self.root = None
        
        # stopping conditions
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    def build_tree(self, dataset, curr_depth=0):
        ''' recursive function to build the tree ''' 
        
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        
        # split until stopping conditions are met
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            # find the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            # check if information gain is positive
            if best_split["info_gain"]>0:
                # recur left
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                # recur right
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                # return decision node
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["info_gain"])
        
        # compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        # return leaf node
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        ''' function to find the best split '''
        
        # dictionary to store the best split
        best_split = {}
        max_info_gain = -float("inf")
        
        # loop over all the features
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            # loop over all the feature values present in the data
            for threshold in possible_thresholds:
                # get current split
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # check if childs are not null
                if len(dataset_left)>0 and len(dataset_right)>0:
                    print("current split value is",threshold)
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # compute information gain
                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")
                    # update the best split if needed
                    print("current curr_info_gain is ",curr_info_gain)
                    if curr_info_gain>max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
                        
        # return best split
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        ''' function to split the data '''
        
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    def information_gain(self, parent, l_child, r_child, mode="entropy"):
        ''' function to compute information gain '''
        
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode=="gini":
            gain = self.gini_index(parent) - (weight_l*self.gini_index(l_child) + weight_r*self.gini_index(r_child))
        else:
            gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
        return gain
    
    def entropy(self, y):
        ''' function to compute entropy '''
        
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy
    
    def gini_index(self, y):
        ''' function to compute gini index '''
        
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            gini += p_cls**2
        return 1 - gini
        
    def calculate_leaf_value(self, Y):
        ''' function to compute leaf node '''
        
        Y = list(Y)
        return max(Y, key=Y.count)
    
    def print_tree(self, tree=None, indent=" "):
        ''' function to print the tree '''
        
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    def fit(self, X, Y):
        ''' function to train the tree '''
        
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
    
    def predict(self, X):
        ''' function to predict new dataset '''
        
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions
    
    def make_prediction(self, x, tree):
        ''' function to predict a single data point '''
        
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]

        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)

In [160]:
X

array([[-1.9584  ],
       [-1.9582  ],
       [-1.8824  ],
       [-1.8029  ],
       [-1.7062  ],
       [-1.6576  ],
       [-1.5978  ],
       [-1.5791  ],
       [-1.5228  ],
       [-1.4856  ],
       [-1.4774  ],
       [-1.4419  ],
       [-1.4328  ],
       [-1.417   ],
       [-1.3959  ],
       [-1.3787  ],
       [-1.3786  ],
       [-1.3729  ],
       [-1.2962  ],
       [-1.2739  ],
       [-1.2425  ],
       [-1.2354  ],
       [-1.2122  ],
       [-1.1936  ],
       [-1.1806  ],
       [-1.0698  ],
       [-1.055   ],
       [-1.0396  ],
       [-1.0276  ],
       [-1.0139  ],
       [-1.0054  ],
       [-1.0019  ],
       [-0.98442 ],
       [-0.96189 ],
       [-0.96085 ],
       [-0.91535 ],
       [-0.91403 ],
       [-0.90936 ],
       [-0.9075  ],
       [-0.88789 ],
       [-0.88104 ],
       [-0.86176 ],
       [-0.84763 ],
       [-0.84698 ],
       [-0.81171 ],
       [-0.80569 ],
       [-0.78022 ],
       [-0.77513 ],
       [-0.77022 ],
       [-0.76553 ],


In [159]:
classifier = DecisionTreeClassifier(min_samples_split=2, max_depth=6)
classifier.fit(X,y)
classifier.print_tree()

current split value is -1.9584
current curr_info_gain is  0.003786827180967678
current split value is -1.9582
current curr_info_gain is  0.007630599131573879
current split value is -1.8824
current curr_info_gain is  0.011532610051128711
current split value is -1.8029
current curr_info_gain is  0.015494193656478195
current split value is -1.7062
current curr_info_gain is  0.019516724701910126
current split value is -1.6576
current curr_info_gain is  0.02360162056975168
current split value is -1.5978
current curr_info_gain is  0.027750342935528194
current split value is -1.5791
current curr_info_gain is  0.03196439951178953
current split value is -1.5228
current curr_info_gain is  0.03624534587497552
current split value is -1.4856
current curr_info_gain is  0.040594787379972797
current split value is -1.4774
current curr_info_gain is  0.045014381167308326
current split value is -1.4419
current curr_info_gain is  0.04950583826825933
current split value is -1.4328
current curr_info_gain is

In [133]:
X_test=pd.read_csv("PS4-1/PS4_2_X_Test.csv")
X_test = X_test.iloc[:, :].values
Y_test=pd.read_csv("PS4-1/PS4_2_Y_Test.csv")
y = Y_test.iloc[:, -1].values.reshape(-1,1)
Y_pred = classifier.predict(X_test) 
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, Y_pred)

1.0